In [1]:
import torchvision
import torch.nn

In [2]:
#model = torchvision.models.squeezenet1_1()
#model.features[0] = torch.nn.Conv2d(1, 64, kernel_size=(3, 3), stride=(2, 2))
#model.classifier[1] = torch.nn.Conv2d(512, 4, kernel_size=(1, 1), stride=(1, 1))
#model.classifier[3] = torch.nn.Identity()

#model.num_classes = 4

In [3]:
dummy_img = torch.rand([8,1,28,28])

In [4]:
#model(dummy_img)

In [5]:
model_kwargs = {
    'in_channels' : 1, # not in use for decentnet, but decenbaseline
    'n_classes': 4, # filled in the dataset
    'out_dim' :  [1, 8, 16, 32], # [1, 8, 16, 32], #[1, 16, 24, 32] # entry, decent1, decent2, decent3
    'grid_size' : 18*18,
    'criterion' : torch.nn.CrossEntropyLoss(), # torch.nn.BCEWithLogitsLoss(),
    'new_cc_mode' : True, # this is for using the new connection cost loss term
    'reset_optimiser_at_update' : True, # needs to be resetted when pruning # not needed anymore
    'optimizer': "sgd", # sgd adamw
    'base_lr': 0.1, #0.001,
    'min_lr' : 0.001, #0.00001,
    'momentum' : 0.9,
    'lr_update' : 100,
    # decentnet
    'cc_weight': 5, # high weight as the cc doesn't change a lot
    'cc_metric' : 'l2', # connection cost metric (for loss) - distance metric # no idea how the torch works oops
    'ci_metric' : 'l2', # todo: should be l2 # channel importance metric (for pruning)
    'cm_metric' : 'not implemented yet', # 'count', # crossing minimisation 
    'update_every_nth_epoch' : 3, # 5 # todo - remove from code
    'pretrain_epochs' : 3, # 20 # todo - remove from code
    'prune_keep' : 0.95, # 0.97, # in each epoch
    'prune_keep_total' : 0.4, # this number is not exact, depends on the prune_keep value
}


In [142]:
# =============================================================================
# alphabetic order misc
# =============================================================================
import math
import matplotlib.pyplot as plt
import os
import pandas as pd
# =============================================================================
# torch
# =============================================================================
import torch
import torch.nn as nn
import torchvision # for model
import torch.nn # for torch.nn.Identity


class DecentBaseline(nn.Module):
    def __init__(self, model_kwargs, log_dir="", ckpt_path='') -> None:
        super(DecentBaseline, self).__init__()
        
        self.n_classes = model_kwargs["n_classes"]  
        self.in_channels = model_kwargs["in_channels"]  
        
                          
        # backbone
        #self.model = torchvision.models.alexnet()
                  # Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        #self.model.features[0][0] = torch.nn.Conv2d(self.in_channels, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        # self.model.features[0] = torch.nn.Conv2d(self.in_channels, 64, kernel_size=(3, 3), stride=(2, 2))
        
        # self.model.classifier[1] = torch.nn.Conv2d(512, self.n_classes, kernel_size=(3, 3), stride=(1, 1))
        
        #self.model.classifier = torch.nn.Identity()

        # self.model.num_classes = self.n_classes
        
        #self.model.avgpool = torch.nn.Identity()
        #self.model.classifier = torch.nn.Identity()
        
        """
          (avgpool): AdaptiveAvgPool2d(output_size=1)
          (classifier): Sequential(
            (0): Dropout(p=0.2, inplace=True)
            (1): Linear(in_features=1280, out_features=1000, bias=True)
          )
          """
        
        # self.model.classifier = torch.nn.Identity() # replace last layer with "nothing"
        #self.model.classifier[1] = 
        #self.model.classifier[2] = torch.nn.Identity() # remove relu??
        #self.model.classifier[3] = torch.nn.Identity() # replace last layer with "nothing"
        
        #self.model.classifier = torch.nn.Identity()
        
        self.e_b0 = torchvision.models.efficientnet_b0()
        self.e_b0.features[0][0] = torch.nn.Conv2d(self.in_channels, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        
        #self.alex = torchvision.models.alexnet()
        #self.alex.features[0] = torch.nn.Conv2d(self.in_channels, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
        #self.alex.avgpool = torch.nn.Identity()
        #self.alex.classifier = torch.nn.Identity()
        #self.alex.features[10] = torch.nn.Conv2d(256, 4, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        
        output_size = torchvision.models.efficientnet_b0().features[-1][1].num_features 
        
        self.random_conv = torch.nn.Conv2d(output_size, self.n_classes, kernel_size=(1, 1), stride=(1, 1))
        
        #self.classifier2 = torch.nn.Conv2d(1000, self.n_classes, kernel_size=(3, 3), stride=(1, 1))
        
    def forward(self, x, mode=None): # normal image, not a real x
        
        x = self.e_b0.features(x) # self.alex.features(x)
        x = self.random_conv(x)        
        x = torch.nn.functional.max_pool2d(x, kernel_size=x.size()[2:]) # global max pooling
        x = x.reshape(x.size(0), -1) # respahe or flatten
        
        return x
    
    




In [152]:
torchvision.models.efficientnet_b0().features[-1][1].num_features 

1280

In [143]:
model2 = DecentBaseline(model_kwargs)

In [144]:
model2(dummy_img).shape

torch.Size([8, 1, 224, 224])
size torch.Size([8, 1280, 7, 7])
size torch.Size([8, 4, 7, 7])


torch.Size([8, 4])

In [71]:
alex = torchvision.models.alexnet()
alex.avgpool = torch.nn.Identity()
alex.classifier = torch.nn.Identity()

In [72]:
alex

AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (avgpool): Identity()
  (classifier): Identity()
)

In [10]:
torchvision.models.efficientnet_b0()

EfficientNet(
  (features): Sequential(
    (0): Conv2dNormActivation(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): SiLU(inplace=True)
    )
    (1): Sequential(
      (0): MBConv(
        (block): Sequential(
          (0): Conv2dNormActivation(
            (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
            (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (2): SiLU(inplace=True)
          )
          (1): SqueezeExcitation(
            (avgpool): AdaptiveAvgPool2d(output_size=1)
            (fc1): Conv2d(32, 8, kernel_size=(1, 1), stride=(1, 1))
            (fc2): Conv2d(8, 32, kernel_size=(1, 1), stride=(1, 1))
            (activation): SiLU(inplace=True)
            (scale_activation): Sigmoid()
          )
          (2): Conv2dNormActivat

In [14]:
torchvision.models.squeezenet1_1()

SqueezeNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
    (3): Fire(
      (squeeze): Conv2d(64, 16, kernel_size=(1, 1), stride=(1, 1))
      (squeeze_activation): ReLU(inplace=True)
      (expand1x1): Conv2d(16, 64, kernel_size=(1, 1), stride=(1, 1))
      (expand1x1_activation): ReLU(inplace=True)
      (expand3x3): Conv2d(16, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (expand3x3_activation): ReLU(inplace=True)
    )
    (4): Fire(
      (squeeze): Conv2d(128, 16, kernel_size=(1, 1), stride=(1, 1))
      (squeeze_activation): ReLU(inplace=True)
      (expand1x1): Conv2d(16, 64, kernel_size=(1, 1), stride=(1, 1))
      (expand1x1_activation): ReLU(inplace=True)
      (expand3x3): Conv2d(16, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (expand3x3_activation): ReLU(inplace=True)
    )
    (5): MaxPool2d